# 第10天：分类算法进阶实战 🚀

## 🎯 今日目标
1. **回顾第9天练习题** - 检查理解程度
2. **掌握交叉验证** - 评估模型稳定性
3. **学习模型调优** - GridSearchCV参数优化
4. **实战新数据集** - 乳腺癌诊断项目
5. **模型对比分析** - 深入理解算法差异

## 📚 今日重点
- 练习题解答与深入理解
- 交叉验证与模型评估
- 超参数调优技术
- 实际医疗诊断项目
- 特征工程与可视化

# 1. 第9天练习题回顾与解答 📝

## 1.1 基础理解题解答

**Q1: 分类与回归的主要区别？**

**答案：**
- **分类**：输出离散类别标签（如：垃圾邮件/正常邮件）
- **回归**：输出连续数值（如：房价预测）
- **评估指标**：分类用准确率、精确率、召回率；回归用MSE、R²

**Q2: 逻辑回归为什么用sigmoid函数？**

**答案：**
- 将线性输出压缩到(0,1)区间，表示概率
- 便于优化，有良好的数学性质（可导、单调）
- 输出可以解释为样本属于正类的概率

**Q3: 决策树如何选择分裂特征？**

**答案：**
- **信息增益**：选择能最大程度减少不确定性的特征
- **基尼指数**：选择能最大程度减少不纯度的特征
- **目标**：让子节点尽可能纯净（同一类别）

**Q4: 混淆矩阵如何解读？**

**答案：**
- **对角线**：正确分类的样本数
- **非对角线**：错误分类的样本数
- **精确率** = TP / (TP + FP)
- **召回率** = TP / (TP + FN)

**Q5: 逻辑回归vs决策树优缺点？**

**答案：**
- **逻辑回归**：训练快、不易过拟合、可解释性强，但只能处理线性关系
- **决策树**：可处理非线性关系、特征重要性明确，但容易过拟合

In [ ]:
# 导入必要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    precision_score, recall_score, f1_score, roc_curve, auc
)
from sklearn.datasets import load_iris, load_breast_cancer
from sklearn.preprocessing import StandardScaler

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

print("✅ 库导入成功！")

# 2. 交叉验证与模型稳定性评估 📊

## 2.1 为什么需要交叉验证？

**问题**：只用一次train_test_split评估模型可靠吗？

**答案**：不可靠！因为：
- 数据划分的随机性会影响结果
- 可能过拟合到特定的训练/测试集
- 无法评估模型的泛化能力

**解决方案**：交叉验证（Cross-Validation）
- 多次划分数据，取平均性能
- 更准确地评估模型稳定性
- 减少随机性影响

In [ ]:
# 加载鸢尾花数据集
iris = load_iris()
X = iris.data
y = iris.target

print(f"数据集形状: X={X.shape}, y={y.shape}")
print(f"特征名称: {iris.feature_names}")
print(f"类别名称: {iris.target_names}")

# 对比：单次划分 vs 交叉验证
print("\n🔍 单次划分结果:")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

logreg = LogisticRegression(max_iter=1000, random_state=42)
logreg.fit(X_train, y_train)
single_score = logreg.score(X_test, y_test)
print(f"单次划分准确率: {single_score:.4f}")

# 交叉验证
print("\n📊 交叉验证结果:")
cv_scores = cross_val_score(logreg, X, y, cv=5)
print(f"5折交叉验证分数: {cv_scores}")
print(f"平均准确率: {cv_scores.mean():.4f}")
print(f"标准差: {cv_scores.std():.4f}")
print(f"95%置信区间: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

# 可视化交叉验证结果
plt.figure(figsize=(10, 6))
plt.subplot(1, 2, 1)
plt.bar(['单次划分', '交叉验证'], [single_score, cv_scores.mean()])
plt.ylabel('准确率')
plt.title('单次划分 vs 交叉验证')
plt.ylim(0.8, 1.0)

plt.subplot(1, 2, 2)
plt.plot(range(1, 6), cv_scores, 'o-', linewidth=2, markersize=8)
plt.xlabel('折数')
plt.ylabel('准确率')
plt.title('5折交叉验证分数变化')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 3. 超参数调优实战 🔧

## 3.1 GridSearchCV参数优化

**问题**：如何找到最优的超参数？

**解决方案**：网格搜索（Grid Search）
- 遍历所有参数组合
- 使用交叉验证评估
- 选择最佳参数组合

In [ ]:
# 逻辑回归参数调优
print("🔧 逻辑回归参数调优:")
param_grid_logreg = {
    'C': [0.01, 0.1, 1, 10, 100],  # 正则化强度
    'penalty': ['l1', 'l2'],         # 正则化类型
    'solver': ['liblinear']          # 优化算法
}

grid_logreg = GridSearchCV(
    LogisticRegression(max_iter=1000, random_state=42),
    param_grid_logreg, cv=5, scoring='accuracy', n_jobs=-1
)
grid_logreg.fit(X, y)

print(f"最优参数: {grid_logreg.best_params_}")
print(f"最优交叉验证分数: {grid_logreg.best_score_:.4f}")
print(f"参数搜索空间大小: {len(grid_logreg.cv_results_['params'])}")

# 决策树参数调优
print("\n🌳 决策树参数调优:")
param_grid_tree = {
    'max_depth': [2, 3, 4, 5, 6, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_tree = GridSearchCV(
    DecisionTreeClassifier(random_state=42),
    param_grid_tree, cv=5, scoring='accuracy', n_jobs=-1
)
grid_tree.fit(X, y)

print(f"最优参数: {grid_tree.best_params_}")
print(f"最优交叉验证分数: {grid_tree.best_score_:.4f}")

# 可视化调优结果
plt.figure(figsize=(12, 5))

# 逻辑回归调优结果
plt.subplot(1, 2, 1)
results_logreg = pd.DataFrame(grid_logreg.cv_results_)
best_logreg = results_logreg[results_logreg['rank_test_score'] == 1].iloc[0]
print(f"\n逻辑回归调优提升: {best_logreg['mean_test_score'] - cv_scores.mean():.4f}")

# 决策树调优结果
plt.subplot(1, 2, 2)
results_tree = pd.DataFrame(grid_tree.cv_results_)
best_tree = results_tree[results_tree['rank_test_score'] == 1].iloc[0]
print(f"决策树调优提升: {best_tree['mean_test_score'] - cv_scores.mean():.4f}")

plt.tight_layout()
plt.show()

# 4. 实战项目：乳腺癌诊断 🏥

## 4.1 数据集介绍

**乳腺癌数据集特点：**
- 569个样本，30个特征
- 二分类问题：恶性(0) vs 良性(1)
- 特征为细胞核的测量值
- 医疗诊断的经典数据集

**为什么选择这个数据集？**
- 特征数量多，需要特征工程
- 医疗诊断，对模型性能要求高
- 数据不平衡，需要特殊处理

In [ ]:
# 加载乳腺癌数据集
cancer = load_breast_cancer()
X_cancer = cancer.data
y_cancer = cancer.target

print(f"乳腺癌数据集形状: X={X_cancer.shape}, y={y_cancer.shape}")
print(f"类别分布: {np.bincount(y_cancer)}")
print(f"类别含义: 0=恶性, 1=良性")
print(f"特征数量: {X_cancer.shape[1]}")

# 数据预处理 - 特征缩放
print("\n🔧 数据预处理:")
scaler = StandardScaler()
X_cancer_scaled = scaler.fit_transform(X_cancer)

print(f"原始数据范围: [{X_cancer.min():.2f}, {X_cancer.max():.2f}]")
print(f"缩放后范围: [{X_cancer_scaled.min():.2f}, {X_cancer_scaled.max():.2f}]")

# 划分数据集
X_train_cancer, X_test_cancer, y_train_cancer, y_test_cancer = train_test_split(
    X_cancer_scaled, y_cancer, test_size=0.2, random_state=42, stratify=y_cancer
)

print(f"\n训练集大小: {X_train_cancer.shape[0]}")
print(f"测试集大小: {X_test_cancer.shape[0]}")

# 训练模型
print("\n🏥 模型训练与评估:")
logreg_cancer = LogisticRegression(random_state=42)
dtree_cancer = DecisionTreeClassifier(max_depth=5, random_state=42)

logreg_cancer.fit(X_train_cancer, y_train_cancer)
dtree_cancer.fit(X_train_cancer, y_train_cancer)

# 预测
y_pred_logreg_cancer = logreg_cancer.predict(X_test_cancer)
y_pred_dtree_cancer = dtree_cancer.predict(X_test_cancer)

# 评估
print(f"逻辑回归准确率: {accuracy_score(y_test_cancer, y_pred_logreg_cancer):.4f}")
print(f"决策树准确率: {accuracy_score(y_test_cancer, y_pred_dtree_cancer):.4f}")

print("\n逻辑回归详细评估:")
print(classification_report(y_test_cancer, y_pred_logreg_cancer, 
                          target_names=['恶性', '良性']))

print("\n决策树详细评估:")
print(classification_report(y_test_cancer, y_pred_dtree_cancer, 
                          target_names=['恶性', '良性']))

# 5. 特征工程与可视化 📊

## 5.1 特征重要性分析

在医疗诊断中，了解哪些特征最重要非常关键！

In [ ]:
# 特征重要性分析
print("🔍 特征重要性分析:")

# 逻辑回归特征重要性（系数绝对值）
logreg_importance = np.abs(logreg_cancer.coef_[0])
feature_names = cancer.feature_names

# 获取前10个最重要的特征
top_indices = np.argsort(logreg_importance)[-10:]
top_features = [feature_names[i] for i in top_indices]
top_importance = logreg_importance[top_indices]

print("\n逻辑回归 - 前10个最重要特征:")
for name, importance in zip(top_features, top_importance):
    print(f"  {name}: {importance:.4f}")

# 决策树特征重要性
tree_importance = dtree_cancer.feature_importances_
top_tree_indices = np.argsort(tree_importance)[-10:]
top_tree_features = [feature_names[i] for i in top_tree_indices]
top_tree_importance = tree_importance[top_tree_indices]

print("\n决策树 - 前10个最重要特征:")
for name, importance in zip(top_tree_features, top_tree_importance):
    print(f"  {name}: {importance:.4f}")

# 可视化特征重要性
plt.figure(figsize=(15, 6))

plt.subplot(1, 2, 1)
plt.barh(range(len(top_features)), top_importance)
plt.yticks(range(len(top_features)), top_features)
plt.xlabel('重要性')
plt.title('逻辑回归特征重要性（前10）')

plt.subplot(1, 2, 2)
plt.barh(range(len(top_tree_features)), top_tree_importance)
plt.yticks(range(len(top_tree_features)), top_tree_features)
plt.xlabel('重要性')
plt.title('决策树特征重要性（前10）')

plt.tight_layout()
plt.show()

# 特征相关性分析
print("\n📊 特征相关性分析:")
df_cancer = pd.DataFrame(X_cancer_scaled, columns=cancer.feature_names)
df_cancer['target'] = y_cancer

# 计算与目标的相关性
correlations = df_cancer.corr()['target'].abs().sort_values(ascending=False)
print("\n与目标变量相关性最高的特征:")
print(correlations.head(10))

# 6. 模型对比与总结 📈

## 6.1 综合性能对比

让我们对比两个模型在不同指标上的表现：

In [ ]:
# 综合性能对比
print("📊 模型性能综合对比:")

# 计算各种指标
def calculate_metrics(y_true, y_pred, model_name):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    print(f"\n{model_name}:")
    print(f"  准确率: {accuracy:.4f}")
    print(f"  精确率: {precision:.4f}")
    print(f"  召回率: {recall:.4f}")
    print(f"  F1分数: {f1:.4f}")
    
    return [accuracy, precision, recall, f1]

# 计算指标
logreg_metrics = calculate_metrics(y_test_cancer, y_pred_logreg_cancer, "逻辑回归")
tree_metrics = calculate_metrics(y_test_cancer, y_pred_dtree_cancer, "决策树")

# 可视化对比
metrics_names = ['准确率', '精确率', '召回率', 'F1分数']
x = np.arange(len(metrics_names))
width = 0.35

plt.figure(figsize=(10, 6))
plt.bar(x - width/2, logreg_metrics, width, label='逻辑回归', color='skyblue')
plt.bar(x + width/2, tree_metrics, width, label='决策树', color='lightcoral')

plt.xlabel('评估指标')
plt.ylabel('分数')
plt.title('模型性能对比')
plt.xticks(x, metrics_names)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 交叉验证对比
print("\n📈 交叉验证稳定性对比:")
cv_logreg = cross_val_score(logreg_cancer, X_cancer_scaled, y_cancer, cv=5)
cv_tree = cross_val_score(dtree_cancer, X_cancer_scaled, y_cancer, cv=5)

print(f"逻辑回归CV: {cv_logreg.mean():.4f} (+/- {cv_logreg.std() * 2:.4f})")
print(f"决策树CV: {cv_tree.mean():.4f} (+/- {cv_tree.std() * 2:.4f})")

# 可视化交叉验证结果
plt.figure(figsize=(8, 6))
plt.boxplot([cv_logreg, cv_tree], labels=['逻辑回归', '决策树'])
plt.ylabel('交叉验证准确率')
plt.title('模型稳定性对比')
plt.grid(True, alpha=0.3)
plt.show()

# 7. 进阶练习与思考题 💭

## 7.1 编程练习题

1. **特征选择实验**：只用前5个最重要的特征训练模型，观察性能变化
2. **参数调优**：对乳腺癌数据集进行更深入的GridSearchCV调优
3. **ROC曲线**：为乳腺癌数据集绘制ROC曲线，比较两个模型
4. **数据不平衡**：如果恶性样本只有10%，如何处理？

## 7.2 思考题

1. **为什么乳腺癌数据集需要特征缩放？**
2. **在医疗诊断中，精确率和召回率哪个更重要？为什么？**
3. **如何解释决策树在乳腺癌数据集上的特征重要性？**
4. **如果数据不平衡，应该如何处理？**
5. **交叉验证为什么比单次划分更可靠？**

## 7.3 实战挑战

尝试用其他数据集（如手写数字数据集）进行多分类实验，并总结不同算法的适用场景。

---
**今日总结**：你已经掌握了分类算法的进阶应用！明天我们将学习更高级的算法。